In [1]:
import os
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [2]:
try:
    connection = mysql.connector.connect(
        host='127.0.0.1',
        port=3307,
        database=os.getenv('MYSQL_DATABASES'),
        user=os.getenv('MYSQL_ROOT_USER'),
        password=os.getenv('MYSQL_ROOT_PASSWORD')
    )
except Error as e:
    print("Lỗi kết nối MySQL:", e)
    exit()

In [13]:
folder_path = '../data/'

In [17]:
file_path = os.path.join(folder_path, 'companies.csv')
df = pd.read_csv(file_path)
cursor = connection.cursor()
for _, row in df.iterrows():
    query1 = "INSERT INTO companies (symbol, comGroupCode, organName, organShortName, organTypeCode, icbName, sector, industry, `group`, subGroup, icbCode) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    value1 = (row['ticker'], row['comGroupCode'],
              row['organName'], row['organShortName'], row['organTypeCode'], row['icbName'],
              row['sector'], row['industry'], row['group'], row['subgroup'], row['icbCode'])
    cursor.execute(query1, value1)
        
connection.commit()
cursor.close()

True

In [18]:
file_path = os.path.join(folder_path, 'all_stocks.csv')
df = pd.read_csv(file_path)
df = df.drop(["Time"], axis=1)
# df.head()
cursor = connection.cursor()
for _, row in df.iterrows():
    query2 = """
        INSERT INTO Trades (symbol, value, tradingDate, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    value2 = (row['Symbol'], row['Value'], row['TradingDate'], row['Open'], row['High'],
              row['Low'], row['Close'], row['Volume'])
    cursor.execute(query2, value2)
    
connection.commit()
cursor.close()

True

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.sql.expression import select, text

engine = create_engine('trino://trino@localhost:8085/iceberg')
connection = engine.connect()

rows = connection.execute(text("SELECT * FROM information_schema.tables")).fetchall()

In [ ]:
for i in rows:
    print(i)

In [3]:
import pandas as pd
import os, glob

In [4]:
path = "../data/"

In [ ]:
file_path = os.path.join(path, 'companies.csv')
file_path

In [ ]:
dff = pd.read_csv(file_path, index_col=None, header=0)
dff.head()

In [11]:
all_files = glob.glob(os.path.join(path, "*.csv"))
# all_files
l = []
for filename in all_files:
    if filename != "../data\\companies.csv":
        df = pd.read_csv(filename, index_col=None, header=0)
        l.append(df)

frame = pd.concat(l, ignore_index=True, axis=0)
# frame['TradingDate'] = pd.to_datetime(frame['TradingDate'], dayfirst=True)
# frame = frame.drop(["Unnamed: 0"], axis=1)
frame.head()

ValueError: time data "2013-01-02" doesn't match format "%d/%m/%Y", at position 2706. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [42]:
frame.to_csv('../data/all_stocks.csv', index=False)